In [1]:
%run Structure.ipynb

# this is hamilton's quaternion product
def product(a, b):
    v = b[0] * a[1:] + a[0] * b[1:] + np.cross(a[1:], b[1:])
    return np.array([a[0] * b[0] - np.dot(a[1:], b[1:]), v[0], v[1], v[2]])

# this is the inverse of a unit quat
def conjugate(q):
    return np.array([q[0], -q[1], -q[2], -q[3]])

# this rotates a vector with a fixed frame
def sandwich(q, v):
    return product(q, product(np.array([0, v[0], v[1], v[2]]), conjugate(q)))[1:]

# this rotates a frame with a fixed vector
def frame_rotation(q, v):
    return sandwich(conjugate(q), v)

# this constrains a quat to S3
def renormalize(q):
    return q / np.linalg.norm(q)

In [10]:

class Environment():
    def __init__(self, latitude=32):
        self.ka = 1.4                   # Ratio of specific heats, air  
        self.Ra = 287.1                 # Avg. specific gas constant (dry air)
        self.latitude = np.radians(latitude) # degrees north, launch site
        # International Gravity Formula (IGF) 1980, Geodetic Reference System 1980 (GRS80)
        self.IGF = 9.780327 * (1 + 0.0053024 * np.sin(self.latitude)**2 - 0.0000058 * np.sin(2*self.latitude)**2)
        # Free Air Correction (FAC)
        self.FAC = -3.086 * 10**(-6)
    # https://www.sensorsone.com/local-gravity-calculator/
    def g_accel(self, x):
        return self.IGF + self.FAC * x[2]
    # U.S. 1976 Standard Atmosphere, pls sanity check eventually
    def std_at(self, x):
        if x[2] < 11000:
            T = 15.04 - 0.00649*x[2]
            p = 101.29*((T + 273.1)/288.08)**5.256
    
        elif 11000 <= x[2] and x[2] <25000:
            T = -56.46
            p = 22.65*np.exp(1.73 - 0.000157*x[2])
    
        else:
            T = -131.21 + 0.00299*x[2]
            p = 2.488 * ((T + 273.1)/216.6)**(-11.388)
    
        p_a = p*1000                 # Ambient air pressure [Pa]
        rho = p/(0.2869*(T + 273.1)) # Ambient air density [kg/m^3]
        T_a = T + 273.1              # Ambient air temperature [K]
        return (p_a, rho, T_a)
    # calculates drag force, etc
    def aero(self, x, v, C_d, A, rho, T_a):
        # Check Knudsen number and consider other drag models (e.g. rarified gas dyn vs. quadratic drag)
        Ma = np.linalg.norm(v) / np.sqrt(self.ka * self.Ra * T_a)
        q = 0.5 * rho * np.linalg.norm(v)**2         # Dynamic pressure [Pa]
        D = q * C_d * A                              # Drag force       [N]
        return (D, q, Ma)

# state = [m, x, q, v, w], is it an issue that our CoM changes but we still use F=MA?
# parameters = [mdot, thtl, I_body, forces, torques], is it a sin to not update MoI as a state variable?
# reference (7.233) on page 435 of Shabana for dynamical equations
def ODE(state, parameters):
    def dm_dt(mdot, throttle):
        return -mdot*throttle
    # kinematic equations of motion
    def dx_dt(v):
        return v
    def dq_dt(q, w):
        return 1/2 * product(q, np.array([0, w[0], w[1], w[2]]))
    # dynamical equations of motion
    def dv_dt(q, mass, forces):
        return sandwich(q, [0,0,1]) * (forces[0] - forces[1])/mass - np.array([0,0,forces[2]])
    def dw_dt(I_body, w, torques):
        return np.linalg.inv(I_body).dot(sum(torques) - np.cross(w, I_body.dot(w)))
    return np.array([dm_dt(parameters[0], parameters[1]),
            dx_dt(state[3]),
            dq_dt(state[2], state[4]),
            dv_dt(state[2], state[0], parameters[3]),
            dw_dt(parameters[2], state[4], parameters[4])])

# get Cd from barrowman
def Forces(env, rkt, state, C_d):
    air = env.std_at(state[1])
    grav = env.g_accel(state[1])
    A = rkt.frontal_area
    aero = env.aero(state[1], state[3], C_d, A, air[1], air[2])
    if rkt.has_fuel():
        throttle = rkt.parts[1].parts[0].throttle_engine(aero[0])
        thrust = rkt.parts[1].parts[0].thrust(air[0], throttle)
    else:
        throttle = 0.
        thrust = 0.
    return ([thrust, aero[0], grav], throttle, air, aero)

# gravitational torque. inputs in inertial frame, outputs in body frame
def T_gg(r_I, attitude_BI, J):
    mu = 3.986004418 * 10**14 # m^3/s^2
    x = r_I # m
    r = np.linalg.norm(x)
    actual = r + 6371000
    n = frame_rotation(attitude_BI, - x / r)
    return (3 * mu / actual**3) * np.cross(n, J.dot(n)) # kg * m^2 / s^2

def Torques(state, parameters):
    T= T_gg(state[1], state[2], parameters)
    return [T]
    #return [np.zeros(3)]

# parameters = [C_d, T_param]
def runge_kutta(env, rkt, state, parameters, dt):
    mdot = rkt.parts[1].parts[0].mdot if rkt.has_fuel() else 0
    F1 = Forces(env, rkt, state, parameters[0])
    T1 = Torques(state, parameters[1])
    k1 = ODE(state, [mdot, F1[1], rkt.moment, F1[0], T1])
    
    state_2 = [sum(pair) for pair in zip(state, dt*k1/2)]
    F2 = Forces(env, rkt, state_2, parameters[0])
    T2 = Torques(state_2, parameters[1])
    k2 = ODE(state_2, [mdot, F2[1], rkt.moment, F2[0], T2])
    
    state_3 = [sum(pair) for pair in zip(state, dt*k2/2)]
    F3 = Forces(env, rkt, state_3, parameters[0])
    T3 = Torques(state_3, parameters[1])
    k3 = ODE(state_3, [mdot, F3[1], rkt.moment, F3[0], T3])
    
    state_4 = [sum(pair) for pair in zip(state, dt*k3)]
    F4 = Forces(env, rkt, state_4, parameters[0])
    T4 = Torques(state_4, parameters[1])
    k4 = ODE(state_4, [mdot, F4[1], rkt.moment, F4[0], T4])
    
    return ((k1 + 2*k2 + 2*k3 + k4)/6,
            (F1[1] + 2*F2[1] + 2*F3[1] + F4[1])/6,
            (F1[3][1] + 2*F2[3][1] + 2*F3[3][1] + F4[3][1])/6,
            (F1[3][2] + 2*F2[3][2] + 2*F3[3][2] + F4[3][2])/6)

def time_step(env, rkt, state, parameters, dt, state_list):
    update = runge_kutta(env, rkt, state, parameters, dt)
    new = state + dt * update[0]
    new[2] = renormalize(new[2])
    print(new[2], new[4])
    state_list.append((new, update[0][3], update[1:]))
    
    del_m_o, del_m_f = proportion(dt*update[0][0], rkt.OF)
    rkt.parts[0].parts[1].drain(del_m_o)
    rkt.parts[0].parts[2].drain(del_m_f)
    rkt.parts[0].sum_parts()
    rkt.sum_parts()

def integration(env, rkt):
    state_list = []
    parameters = [.3, rkt.moment]
    initial_state = np.array([rkt.mass, np.array([0,0,1401]), np.array([1,0,0,0]), np.zeros(3), np.zeros(3)])
    state_list.append((initial_state, 0))
    dt = 0.1
    time_step(env, rkt, state_list[-1][0], parameters, dt, state_list)
    while state_list[-1][0][3][2] > 0:
        time_step(env, rkt, state_list[-1][0], parameters, dt, state_list)
    
    return state_list

def initializeandgo():
    LV4 = create_rocket(123.351, 2.624, 42974,
                    2413166, 3097.82, 1.1251, 23.196,
                    (500,1000), 1.,
                    0.1524, 1.3)
    env = Environment()
    states = integration(env, LV4)
    return LV4, states

lv4, states = initializeandgo()
    

[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]


[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]


[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]


[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]


[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]
[1. 0. 0. 0.] [0. 0. 0.]


In [3]:
print(states[-1][0])

[112.13841084672556
 array([    0.        ,     0.        , 57869.61996173])
 array([1., 0., 0., 0.]) array([ 0.        ,  0.        , -0.75463605])
 array([0., 0., 0.])]


-0.13531304347983125
